In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

* **Data Mining e Machine Learning II** 
* Professor: MARCOS GUIMARÃES
* Aluna: Viviane Silvestre

# **Análises Exploratórias**

1. A análise exploratória serrá realizada para conhecer o conjunto de dados de modo a resumir as características do departamento de crédito de um banco. O objetivo é a automatizar o processo de tomada de decisão para aprovação das linhas de crédito do patrimonio líquido. Assim, seguindo as recomendações da Lei de Igualdade de Oportunidades de Crédito, será criado um modelo de pontuação derivado e estatisticamente sólido baseado nos dados coletados pelas conceções recentes de crédito através do processo atual. O modelo será construído a partir de ferramentas de modelagem preditiva. 

In [ ]:
# Importação do conjunto de dados 
df= pd.read_csv('../input/hmeq-data/hmeq.csv')

In [ ]:
# Verificando o Dataset
df

In [ ]:
# Verificando a imensão do Dataset
print(df.shape)

In [ ]:
# Verificando os tipos de variáveis
df.dtypes


In [ ]:
#Analises de estatísticas descritivas 
print(df.describe().T)

O conjunto de dados possuir em seu maior número 5960 registros, no entanto, algumas variáveis possuem valores nulos, a variável DEBTINC possui 4693, sendo necessário utilizar técnicas para padronizar o conjunto de dados. Que podem ser imputação ou exclusão dos valores missings. 

In [ ]:
# imprimindo os primeiros registros do Dataset
df.head(10)

# **Tratando Missing**

O primeiro passo é verificar a quantidade de missings em cada tipo de variável e assim tomar a decisão de que tipo de técnica será utilizada para padronizar a base de dados. 

In [ ]:
# verificando os valores missing nas variáveis númericas e categóricas
feat_missing = []

for f in df.columns:
    missings = df[f].isnull().sum()
    if missings > 0:
        feat_missing.append(f)
        missings_perc = missings/df.shape[0]
        
        # printing summary of missing values
        print('Variable {} has {} records ({:.2%}) with missing values'.format(f, missings, missings_perc))

# how many variables do present missing values?
print()
print('O total, de {} variáveis com valores missings'.format(len(feat_missing)))

A analise demonstrou que a maior parte dos valores missings está na variável MOTDUE, que é a variável que possui o valor devido pelos clientes nas hipótecas. Como esses valores são muito diferentes, a decisão foi por excluir os valores missings, e assim analisar apenas os valores reais da base de dados. 

In [ ]:
#dropping rows that have missing data
df.dropna(axis=0, how='any', inplace=True)
df.info()


A base de dados padronizada possui 3364 registros e 13 colunas, um conjunto de dados relativamente pequeno para aplicar modelos de predição. Assim vamos criar uma nova coluna que irá verificar se o valor atual da propriedade é maior ou menor que o valor devido da hipoteca. 
VALUE_MORTDUE = Valor da propriedade - valor da hipoteca

In [ ]:
df['VALUE_MORTDUE'] = df['VALUE'] - df['MORTDUE']

In [ ]:
df.info()

# **Análise da Variável - Target (Preditora)**

In [ ]:
# Visualizações 
import seaborn as sns
import matplotlib.pyplot as plt
ax = sns.countplot(y='BAD', data=df).set_title("Situação dos Clientes")

A variável BAD "Target" verifica se o cliente está inadimplente
a resposta 1 = Inadimplete e 0 = Pagamento em Dia
O gráfico demonstrou que a maioria dos clientes estão em dia com o pagamento.

In [ ]:
#Analise do Dataset
import pandas
from pandas.plotting import scatter_matrix
from matplotlib import pyplot

In [ ]:
# Visualização em Gráficos das variáveis categórias
sns.set( rc = {'figure.figsize': (4, 4)})
fcat = ['REASON','JOB']

for col in fcat:
    plt.figure()
    sns.countplot(y=df[col], data=df, palette="RdPu")
    plt.show()

**Análises dos gráficos 
Reason**
1. DebtCon = debt consolidation = Consolidação da Dívida
1. HomeImp = home improvement - Melhoria na casa
A análise demonstrou que a maior parte dos clientes tiveram a dívida consolidade e que apenas 1000 clientes fizeram melhorias no imóvel.

**Job
1. Refere-se a Seix Categorias Ocupacionais** 
Other, Office, Mgr, ProfExe, Sales, Self
A maioria dos clientes não se enquandraram nas categorias e classificaram a ocupação como outras, a maior parte são ProfExe, Office e Mgr. 



In [ ]:
# visualizing numeric variables using seaborn
f, axes = plt.subplots(3,3, figsize=(20,20))
sns.distplot( df["LOAN"] , color="skyblue", bins=15, kde=False, ax=axes[0, 0])
sns.distplot( df["DEBTINC"] , color="olive", bins=15, kde=False, ax=axes[0, 1])
sns.distplot( df["MORTDUE"] , color="orange", bins=15, kde=False, ax=axes[0, 2])
sns.distplot( df["YOJ"] , color="yellow", bins=15, kde=False, ax=axes[1, 0])
sns.distplot( df["VALUE"] , color="pink", bins=15, kde=False, ax=axes[1, 1])
sns.distplot( df["CLAGE"] , color="gold", bins=15, kde=False, ax=axes[1, 2])
sns.distplot( df["CLNO"] , color="teal", bins=15, kde=False, ax=axes[2, 1])
sns.distplot( df['DEROG'], color="blue", bins=15, kde=False, ax=axes[2, 2])
sns.distplot( df['DELINQ'], color="green", bins=15, kde=False, ax=axes[2, 0])

1. **Análises das variáveis númericas - Gráficos de Histograma **
* As variáveis quantitativas demosntram em todos os casos distruíção assimétrica a esquerda.
* Variável LOAN - Os valores dos empréstimos estão concentrados até 20.000,00, com alguns valores extremos acima de 80.000,00
* Variável DEBTINC - que é a razão entre a dívida e o rendimento ou seja a capacidade de pagamento, possui mais concentração entre 30 e 40.
* Variável MORTDUE - que verifica o valor devido da hipóteca, mostra que a maioria dos clientes posseum dívidas que variam entre 40.000 e 80.000
* Variável YOJ - Verifica os anos que trabalha no emprego atual. A maioria trabalha até 5 anos na empresa, alguns clientes trabalham a mais de 40 anos (número fora do padrão)
* Variável VALUE - Que verifica os valores dos imóveis demonstra que a maioria dos imóveis estão avaliados entre 50.000 e 100.000, existindo casos de imóveis acima de 400.000
* Variável CLAGE - verifica a idade em meses da mais da linha de crédito, a concentração está entre 100 e 300 meses (entre 8 e 25 anos) em que adquiriu a primeira linha de crédito. 
* Variável CLNO - verifica quantas linhas de crédito o cliente possui. A maioria possui entre 10 e 25 linhas de crédito. 
* Variável DEROG - quantidade relatórios depreciativos a concentração está entre 1 e 3.
* Variável DELINQ - Quantidade de linhas de crédito inadimplentes concentram-se entre 0 e 3.

* Para as próximas análises será criado variáveis Dummies com objetivo de melhorar a base de dados nas predições que serão realizadas. 

In [ ]:
# Criar variáveis Dummies
Dum_df = pd.get_dummies(df)
print(Dum_df.info())

A variável **VALUE_MORTDUE** foi criada para verificar se existe saldo suficiente sobre o valor do imóvel (VALUE) caso seja necessário executar o valor da hipóteca (MORTDUE).

No gráfico de boxplot observou-se que a média do saldo do imóvel está em torno de 20.000. 
Existem valores extremos negativos, que significa que o valor do imóvel não é suficiente para pagar a hipoteca e valores extremos positivos mostram que os valores dos imóveis é superior ao valor da hipoteca.

In [ ]:
ax = sns.boxplot(x="VALUE_MORTDUE", data=Dum_df)

In [ ]:
import seaborn as sns

sns.scatterplot(data=Dum_df, x='VALUE', y='MORTDUE')

In [ ]:
sns.scatterplot(data=Dum_df, x='VALUE', y='LOAN')

# **Gráficos de Correlação**

In [ ]:
#Base sem Dummies
corr = df.corr()
#Plot figsize
fig, ax = plt.subplots(figsize=(10,8))
#Generate Color Map
colormap = sns.diverging_palette(220, 10, as_cmap=True)
#Generate Heat Map, allow annotations and place floats in map
sns.heatmap(corr, cmap=colormap, annot=True, fmt=".2f")
#Apply xticks
plt.xticks(range(len(corr.columns)), corr.columns);
#Apply yticks
plt.yticks(range(len(corr.columns)), corr.columns)
#show plot
plt.show()

No gráfico de correlação antes da criação das variáveis dummies mostra uma correlação forte entre as variáveis "MORTDUE" e "VALUE" em relação a variável target BAD.

In [ ]:
#Base com Dummies
corr = Dum_df.corr()
#Plot figsize
fig, ax = plt.subplots(figsize=(10,8))
#Generate Color Map
colormap = sns.diverging_palette(220, 10, as_cmap=True)
#Generate Heat Map, allow annotations and place floats in map
sns.heatmap(corr, cmap=colormap, annot=True, fmt=".2f")
#Apply xticks
plt.xticks(range(len(corr.columns)), corr.columns);
#Apply yticks
plt.yticks(range(len(corr.columns)), corr.columns)
#show plot
plt.show()

Após a criação das variáveis dummies percebe-se uma melhora na correlação das variáveis LOAN YOJ, continuando mostrar correlação forte entre VALUE e LOAN.

# **Modelos estatísticos**

Para a aplicação dos modelos estatísticos a base de dados será dividida entre teste e treino. 
1. Serão aplicados os modelos de RandomForestClassifier, cross_val_score, XGBoost e GradientBoostingClassifier. Com o objetivo de verificar se o modelo é capaz de auxiliar na tomada de decisão de conceder ou não o crédito.

In [ ]:
# Dividindo o DataFrame
from sklearn.model_selection import train_test_split

# Treino e teste
train, test = train_test_split(Dum_df, test_size=0.15, random_state=42)

# Veificando o tanho dos DataFrames
train.shape, test.shape



In [ ]:
Dum_df.info()

In [ ]:
# Selecionado as features
feats = [c for c in Dum_df.columns if c not in ['BAD']]

In [ ]:
Dum_df.head()

In [ ]:
# Trabalhando com RandomForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier(n_estimators=200, min_samples_split=5, max_depth=4, random_state=42)
rf.fit(train[feats], train['BAD'])
accuracy_score(test['BAD'], rf.predict(test[feats]))

In [ ]:
# Feature Importance com RF
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
# Usar o cross validation
from sklearn.model_selection import cross_val_score

scores = cross_val_score(rf, train[feats], train['BAD'], n_jobs=-1, cv=5)

scores, scores.mean()

In [ ]:
# Feature Importance com RF
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
# Trabalhando com XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=200, learning_rate=0.09, random_state=42)
xgb.fit(train[feats], train['BAD'])
accuracy_score(test['BAD'], xgb.predict(test[feats]))

In [ ]:
# Feature Importance com RF
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
# Trabalhando com GBM
from sklearn.ensemble import GradientBoostingClassifier
gbm = GradientBoostingClassifier(n_estimators=200, learning_rate=1.0, max_depth=1, random_state=42)
gbm.fit(train[feats], train['BAD'])
accuracy_score(test['BAD'], gbm.predict(test[feats]))

In [ ]:
# Feature Importance com RF
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
print("cross validation - ", scores, scores.mean())
print("RandomForest - ",accuracy_score(test['BAD'], rf.predict(test[feats])))
print("GBM - ",accuracy_score(test['BAD'], gbm.predict(test[feats])))
print("XGBoost - ",accuracy_score(test['BAD'], xgb.predict(test[feats])))

Nas análises de acima o algoritmo que melhor explica o modelo é XGBoost com 95% de acurácia.

Outra metodologia aplicada será a clusterização, o primeiro passo será selecionar as variáveis, para isso considerou o gráfico de correlação. As variáveis escolhidas foram 'MORTDUE', 'LOAN' e 'YOJ'. O objetivo é determinar a quantidade de clusters (grupos) necessários para explicar o modelo. 

# **Clusterização**

In [ ]:
# Determinando a quantidade de clusters

# Importando o k-means
from sklearn.cluster import KMeans

# Selecionando as variaveis para utilizar no modelo.
X= Dum_df[['MORTDUE','LOAN', 'YOJ']]

# Cálculo do SSE - Sum of Squared Erros
sse = []

for k in range (1, 12):
    kmeans = KMeans(n_clusters=k, random_state=42).fit(X)
    sse.append(kmeans.inertia_)
print(sse)

In [ ]:
# Definindo a quantidade clusters utilizando o método Elbow 
import matplotlib.pyplot as plt

plt.plot(range(1, 12), sse, 'bx-')
plt.title('Elbow Method')
plt.xlabel('Número de clusters')
plt.ylabel('SSE')
plt.show()

O gráfico do modelo "Gráfico de Cotovelo" mostrou que o número bom de clusters é 4, uma vez que de 1 a 3 teríamos poucos agrupamentos, ou seja, a informação estaria muito concentrada e de 5 a 6 os agrupamentos estariam muitos dispersos. 

In [ ]:
# Excecutando a clusterização com 4 clusters
kmeans = KMeans(n_clusters=4, init='k-means++', random_state=42)
cluster_id = kmeans.fit_predict(X)
cluster_id

In [ ]:
# Guardar os resultados no dataframe
X['cluster_id'] = cluster_id

#veficando o tamanho do DF
X.sample(10)



In [ ]:
X.head()

Verifica-se que o cluster 2 possui um maior numero de iformações.

In [ ]:
# Plotando os agrupamentos e os centroídes
fig = plt.figure(figsize=(12,8))

plt.scatter(X.values[:,0], X.values[:,1], c=kmeans.labels_)
plt.scatter(kmeans.cluster_centers_[:,0] ,kmeans.cluster_centers_[:,1], color='red', marker="x", s=200)
plt.show()


Os grupos são agrupados por suas similaridades baseado no conceito de distância, a técnica consiste em atribuir um centróide a cada elemento, em seguida, utilizando a distância euclidiana, calcula-se as distâncias de cada elemento, e divide os centróides de acordo com o número de clustes, nesse caso 4.
A partir de então,uUma vez que os pontos foram atribuídos aos clusters conforme sua distância, recalcula-se o valor dos centróide, então é calculada a média dos valores dos pontos de dados de cada cluster e o valor médio será o novo centróide. E a partir do centróide são calculadas as distâncias de cada elemento e agrupando aqueles com maior próximidade. 
Atavés dessa técnica verificamos que os grupos laranja e preto tiveram uma quantidade de elementos com menor distância, tornan-se grupos maiores e dois grupos com quantidade menor de elementos por esses elementos estarem mais distantes dos centróides. 



In [ ]:
Dum_df.info()

# **Modelo de Regressão Logística**


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, classification_report,\
                            accuracy_score, confusion_matrix, auc

#Utilizando o statsmodels
#Verificar as chances de um cliente ser inadimplente em função das variáveis LOAN + YOJ + MORTDUE (Valor do empréstimo, ocupação e o valor da hipoteca)
#BAD = 1 (Cliente Inadimplente)
#BAD = 2 (Cliente adimplente)

modelo = smf.glm(formula='BAD ~ LOAN + YOJ + MORTDUE', data=Dum_df,
                family = sm.families.Binomial()).fit()
print(modelo.summary())

In [ ]:
#gerar osdados em percentuais relativos de chances de inadimplencia
print(np.exp(modelo.params[1:]))

In [ ]:
(np.exp(modelo.params[1:]) -1) * 100


Para a regressão logistica foi utilizada a base com a criação das variáveis dummies. 
Todos os coeficientes foram estatísticamente significativos para o modelo, pois o p-valor foi abaixo do nível de significância de 5%.
A chance de um cliente ser inadimplente é de 99% se for considerado o montande do empréstimo LOAN, 96% está associado a ocupação YOJ e 99% ao valor devido da hipoteca MORTDUE. 


In [ ]:
# Agora vamos fazer com sklearn para aproveitar as métricas
model1 = LogisticRegression(penalty='none', solver='newton-cg')
baseline_df = Dum_df[['LOAN', 'YOJ', 'MORTDUE']].dropna()
y = Dum_df.BAD
X = pd.get_dummies(Dum_df[['LOAN', 'YOJ', 'MORTDUE']], drop_first=True)
print(X)

In [ ]:
model1.fit(X, y)

In [ ]:
#Aplicando a Regressão logística com sklearn
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='none',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(model1.coef_) 

In [ ]:
# Predizendo as probabilidades
yhat = model1.predict_proba(X)


In [ ]:
# manter somente para a classe positiva
yhat = yhat[:, 1] 

In [ ]:
# usando a função do sklearn
confusion_matrix(y, model1.predict(X)) 

In [ ]:
#imprimindo a matriz de confusão
pd.crosstab(y, model1.predict(X))  

In [ ]:
acuracia = accuracy_score(y, model1.predict(X))
print('O modelo obteve %0.4f de acurácia.' % acuracia)